In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
import numpy as np

# ----------------------------------------------------
# 1. LOAD & PREPARE TEXT DATA (using Shakespeare)
# ----------------------------------------------------

# Download Shakespeare dataset
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"
)

# Load text
text = open(path_to_file, "r", encoding="utf-8").read()
print("Length of text:", len(text))

# Get unique characters
unique_chars = sorted(list(set(text)))
vocab_size = len(unique_chars)

char_to_idx = {c: i for i, c in enumerate(unique_chars)}
idx_to_char = {i: c for i, c in enumerate(unique_chars)}

# Encode text → integers
encoded_text = np.array([char_to_idx[c] for c in text])

sequence_length = 100  # Shakespeare benefits from longer context
sequences = []
next_chars = []

for i in range(0, len(encoded_text) - sequence_length):
    seq = encoded_text[i:i + sequence_length]
    next_c = encoded_text[i + sequence_length]
    sequences.append(seq)
    next_chars.append(next_c)

X = np.array(sequences)
y = tf.keras.utils.to_categorical(next_chars, num_classes=vocab_size)

print("Training sequences:", X.shape)

# ----------------------------------------------------
# 2. BUILD LSTM MODEL (follows starter code)
# ----------------------------------------------------
embedding_dim = 256
lstm_units = 512

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=sequence_length),
    LSTM(lstm_units, return_sequences=True),
    LSTM(lstm_units),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

# ----------------------------------------------------
# 3. TRAIN MODEL
# ----------------------------------------------------
EPOCHS = 3
model.fit(X, y, batch_size=64, epochs=EPOCHS)

# ----------------------------------------------------
# 4. TEXT GENERATION FUNCTION
# ----------------------------------------------------
def generate_text(seed_text, length=200, temperature=1.0):
    """
    Generates text using trained LSTM.
    """
    generated = seed_text

    for _ in range(length):
        # Convert seed to integers
        seed_encoded = np.array([char_to_idx[c] for c in seed_text[-sequence_length:]])

        # Pad if too short
        seed_encoded = np.pad(
            seed_encoded,
            (sequence_length - len(seed_encoded), 0),
            mode="constant"
        )

        seed_encoded = np.expand_dims(seed_encoded, axis=0)

        # Predict next character
        preds = model.predict(seed_encoded, verbose=0)[0]

        # Temperature sampling
        preds = np.asarray(preds).astype("float64")
        preds = np.log(preds + 1e-9) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        next_idx = np.random.choice(len(preds), p=preds)
        next_char = idx_to_char[next_idx]

        generated += next_char
        seed_text += next_char

    return generated

# ----------------------------------------------------
# 5. EXAMPLE GENERATED OUTPUTS
# ----------------------------------------------------
print("===== Generated Text (Temp = 0.5) =====")
print(generate_text("Shall I compare thee", temperature=0.5))

print("\n===== Generated Text (Temp = 1.0) =====")
print(generate_text("Shall I compare thee", temperature=1.0))

print("\n===== Generated Text (Temp = 1.5) =====")
print(generate_text("Shall I compare thee", temperature=1.5))




Length of text: 1115394
Training sequences: (1115294, 100)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 1041s 60ms/step - loss: 1.8376
Epoch 2/3
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 1036s 59ms/step - loss: 1.3293
Epoch 3/3
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 1036s 59ms/step - loss: 1.2696
===== Generated Text (Temp = 0.5) =====
Shall I compare thee:
I am ancient time and state and follow'd.

PETRUCHIO:
All this is the day of men are sorrow.

CLARENCE:
I do not be a strength to my son of me.

CLARENCE:
Farewell, come hither, come.

AUTOLYCUS:
I 

===== Generated Text (Temp = 1.0) =====
Shall I compare thee:
And tell you, siely and his worn of fear
A bell, he doth raised aallianton
Than in the child. No, I'll come on him:
And in these child, made a little shall
hear it, and in my bristal men I must now,

===== Generated Text (Temp = 1.5) =====
Shall I compare thee, in why,--
Hare, nories homing? why cut him learncelly
Think thee to his kitw? a' callar
Off; but, leoss--youchen; ore 'tis elsener,--
SciliCuld, be leong witch her adved Richmond?.
Hark looks 